This script helps us filter down the birth-nodes in the required district. The raw birth data obtained from WorldPop will contain data points from other adjacent districts. We use geoapify (https://www.geoapify.com/) api to reverse-geocode (https://www.geoapify.com/reverse-geocoding-api) the lat-long and obtain the district name. Once we have the district names, we filter out the birth-nodes corresponding to required district.

In [ ]:
import os
import sys
from pathlib import Path

In [ ]:
# from geopy.geocoders import Nominatim
import pandas as pd
import os
import time
import json

import requests
from requests.structures import CaseInsensitiveDict

In [ ]:
path_data = Path.cwd().parent / 'data'

In [ ]:
geoapify_api_key = open(path_data / '01 geoapify_api_key.txt', "r").read()

births_raw_df = pd.read_csv(path_data / '01 in_births.csv')

births_raw_df['District'] = pd.Series(dtype = 'object')

Iterate through all locations and obtain the district name. If using this script to geocode locations in other districts, slight modifications might be required to the script due to difference in the response from the api. Print out the 'res' object and adjust code according to requirements.

In [ ]:
for i in range(births_raw_df.shape[0]):
    
    url = f"https://api.geoapify.com/v1/geocode/reverse?lat={births_raw_df.loc[i,'Latitude']}&lon={births_raw_df.loc[i,'Longitude']}&apiKey={geoapify_api_key}"
    headers = CaseInsensitiveDict()
    headers["Accept"] = "application/json"
    resp = requests.get(url, headers=headers)
    res = json.loads(resp.text)
    
    try:
        births_raw_df.loc[i,'District'] = res['features'][0]['properties']['state_district']
    except KeyError:
        births_raw_df.loc[i,'District'] = res['features'][0]['properties']['county']
        
    time.sleep(0.25)

Filter out data for required district

In [ ]:
births_clean_district_df = births_raw_df[births_raw_df['District'] == 'Vikarabad District']

births_clean_district_df = births_clean_district_df.drop(columns= 'District')

In [ ]:
births_raw_df.to_csv(path_data / '01 out_births_cleaned_full.csv', index= False)

births_clean_district_df.to_csv(path_data / '01 out_births_cleaned_district.csv', index = False)